In [ ]:
import numpy as np
import pandas as pd 

In [ ]:
df = pd.read_csv('/kaggle/input/sandp500/all_stocks_5yr.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.info()

In [ ]:
print(df.isnull().sum())

In [ ]:
df_clean = df.dropna()

In [ ]:
print(df_clean.isnull().sum())

In [ ]:
df_clean.duplicated().sum()

In [ ]:
df_clean.describe()

 ### Below: Represents the stock's risk profile, showing that small daily price changes are the norm, and extreme movements are rare.

In [ ]:
#ASE
# Calculate and plot daily returns distribution
df_clean['daily_return'] = df_clean['close'].pct_change()

plt.figure(figsize=(8,5))
df_clean['daily_return'].hist(bins=50, alpha=0.7)
plt.title("Distribution of Daily Returns")
plt.xlabel("Daily Return")
plt.ylabel("Frequency")
plt.show()

In [ ]:
Q1 = df_clean["close"].quantile(0.25)
Q3 = df_clean["close"].quantile(0.75)
IQR = Q3 - Q1

outliers = df_clean[(df_clean["close"] < Q1 - 1.5*IQR) | (df_clean["close"] > Q3 + 1.5*IQR)]

In [ ]:
num_outliers = outliers.shape[0]
total_rows = df_clean.shape[0]

percentage = (num_outliers / total_rows) * 100

print(f"number of outliers: {num_outliers}")
print(f"percentage: {percentage:.2f}%")


In [ ]:
num_cols = df_clean.select_dtypes(include=['float64', 'int64']).columns

for col in num_cols:
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    df_clean[col] = df_clean[col].clip(lower=lower_bound, upper=upper_bound)

In [ ]:
print(df_clean.describe())

In [ ]:
Q1 = df_clean["close"].quantile(0.25)
Q3 = df_clean["close"].quantile(0.75)
IQR = Q3 - Q1

outliers = df_clean[(df_clean["close"] < Q1 - 1.5*IQR) | (df_clean["close"] > Q3 + 1.5*IQR)]

In [ ]:
num_outliers = outliers.shape[0]
total_rows = df_clean.shape[0]

percentage = (num_outliers / total_rows) * 100

print(f"number of outliers: {num_outliers}")
print(f"percentage: {percentage:.2f}%")

In [ ]:
df_clean.head()

In [ ]:
pip install mplfinance

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.plot(df['open'],df['volume'] )
plt.title('Plot with close & volume values')
plt.xlabel("Open values ")
plt.ylabel("Volume Values")
plt.show()

### The plot above shows an extreme concentration of high volume at low Open Price values (below 250), with volume dropping sharply for higher prices, indicating an inverse correlation.



In [ ]:
plt.plot(df_clean['open'],df_clean['volume'] )
plt.title('Plot with close & volume values cleaned')
plt.xlabel("Open values ")
plt.ylabel("Volume Values")
plt.show()

### After likely removing high-price outliers, the plot reveals a persistent high volume across all lower price values (0 to 175), suggesting a cleaning process was applied.



In [ ]:
plt.plot(df['close'],df['volume'] )
plt.title('Plot with close & volume values')
plt.xlabel("Close values ")
plt.ylabel("Volume Values")
plt.show()

### Similar to the Open price, the majority of trading volume is heavily concentrated at low Close Price values (below 250), confirming that high volume activity occurs when the stock price is low.



In [ ]:
plt.plot(df_clean['close'],df_clean['volume'] )
plt.title('Plot with close & volume values cleaned')
plt.xlabel("Close values ")
plt.ylabel("Volume Values")
plt.show()

### After outlier removal, the plot shows uniformly high volume across all remaining low Close Price values (0 to 175), confirming high trading activity in this price range.

In [ ]:
plt.plot(df['close'],df['high'] )
plt.title('Plot with high & volume values')
plt.xlabel("Close values ")
plt.ylabel("High Values")
plt.show()

### The plot displays a perfectly linear, strong positive relationship between Close and High prices, with the relationship breaking down and showing outliers at extremely high price values.

In [ ]:
plt.plot(df_clean['close'],df_clean['high'] )
plt.title('Plot with high & volume values cleaned')
plt.xlabel("Close values ")
plt.ylabel("High Values")
plt.show()

### The relationship remains strongly linear after outlier removal, confirming that the Close price is almost always identical or very close to the High price on any given day.

In [ ]:
plt.plot(df['close'],df['low'] )
plt.title('Plot with close & volume values')
plt.xlabel("Close values ")
plt.ylabel("Low Values")
plt.show()

### The plot shows a near-perfect linear correlation between the Close and Low prices, with the relationship breaking down due to outliers at very high price values.

In [ ]:
plt.plot(df_clean['close'],df_clean['low'] )
plt.title('Plot with close & volume values cleaned')
plt.xlabel("Close values ")
plt.ylabel("Low Values")
plt.show()

### After outlier removal, the relationship remains strongly linear, confirming the Close price is almost always identical or very close to the Low price on a given day.

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot (df['close'] , color = 'red' ,linewidth=.25 )
plt.title('Close Values')
plt.xlabel('Rows')
plt.ylabel('Closing Price ($)')
plt.show()

### The time series plot reveals that the Close price is generally low and stable, punctuated by infrequent, extreme, high-magnitude spikes (outliers) across the observation period.

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot (df_clean['close'] , color = 'orange',  linewidth=.25)
plt.title('Closing Price cleaned')
plt.xlabel('Rows')
plt.ylabel('Closing Price ($)')
plt.grid(True, axis='y', alpha=0.5)
plt.show()

### The cleaned Closing Price shows high day-to-day volatility with values consistently fluctuating between approximately $25 and $175, with no clear long-term trend.

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(df.index, df['open'], color = 'red' , linewidth=.25)
plt.title('Opening Price ', fontsize=16)
plt.xlabel('Rows')
plt.ylabel('Opening Price ($)')
plt.grid(True, axis='y', alpha=0.5)
plt.show()

### The Opening Price time series is highly volatile, characterized by frequent extreme spikes (outliers), reaching values well over $1,500, punctuated by periods of low, stable prices.

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(df_clean.index, df_clean['open'],color ='orange', linewidth=.25)
plt.title('Opening Price cleaned', fontsize=16)
plt.xlabel('Rows')
plt.ylabel('Opening Price ($)')
plt.grid(True, axis='y', alpha=0.5)
plt.show()

### The cleaned Opening Price shows high day-to-day volatility with prices consistently fluctuating between approximately $25 and $175, with no discernible long-term trend.

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(df.index, df['high'], color = 'red' , linewidth=.25)
plt.title('high Price ', fontsize=16)
plt.xlabel('Rows')
plt.ylabel('High Price ($)')
plt.grid(True, axis='y', alpha=0.5)
plt.show()

### The High Price time series is highly volatile, characterized by frequent extreme spikes (outliers), reaching values well over $2,000, interspersed with prolonged periods of low, stable prices.

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(df_clean.index, df_clean['high'],color ='orange', linewidth=.25)
plt.title('high Price ', fontsize=16)
plt.xlabel('Rows')
plt.ylabel('High Price ($)')
plt.grid(True, axis='y', alpha=0.5)
plt.show()

### The cleaned High Price shows high day-to-day volatility with values consistently fluctuating between approximately $25 and $175, with no clear long-term trend.

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(df.index, df['low'], color = 'red' , linewidth=.25)
plt.title('Low Price ', fontsize=16)
plt.xlabel('Rows')
plt.ylabel('Low Price ($)')
plt.grid(True, axis='y', alpha=0.5)
plt.show()

### The Low Price time series is highly volatile, characterized by frequent extreme spikes (outliers), reaching values well over $1,500, interspersed with prolonged periods of low, stable prices.

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(df_clean.index, df_clean['low'], color = 'orange' , linewidth=.25)
plt.title('Low Price cleaned ', fontsize=16)
plt.xlabel('Rows')
plt.ylabel('Low Price ($)')
plt.grid(True, axis='y', alpha=0.5)
plt.show()

### The cleaned Low Price shows high day-to-day volatility with values consistently fluctuating between approximately $0 and $175, with no discernible long-term trend.

In [ ]:
df_clean['Name'].value_counts()

In [ ]:
df_clean['Name'].mode()

In [ ]:
df_stock = df_clean[df_clean['Name'] == 'AAPL'].copy()
# Calculate 50-day and 200-day Moving Averages
df_stock['MA_Last 50 day'] = df_stock['close'].rolling(window=50).mean()

plt.figure(figsize=(14, 7))
# Plot Close Price
plt.plot(df_stock.index, df_stock['close'], label='Close Price', color='gray', alpha=0.6)
# Plot MAs
plt.plot(df_stock.index, df_stock['MA_Last 50 day'], label='50-Day MA', color='orange')

plt.title('AAPL Price with Moving Average', fontsize=16)
plt.xlabel('Rows_for_Last50_Dates')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, axis='y', alpha=0.5)
plt.show()

### The 50-day Moving Average (MA) successfully tracks the short-term trend of the AAPL stock, acting as a dynamic support during periods of strong upward movement.

In [ ]:
df_stock = df_clean[df_clean['Name'] == 'AAL'].copy()
# Calculate 50-day and 200-day Moving Averages
df_stock['MA_Last 50 day'] = df_stock['close'].rolling(window=50).mean()

plt.figure(figsize=(14, 7))
# Plot Close Price
plt.plot(df_stock.index, df_stock['close'], label='Close Price', alpha=0.6)
# Plot MAs
plt.plot(df_stock.index, df_stock['MA_Last 50 day'], label='50-Day MA', color='green')

plt.title('AAL Price with Moving Average', fontsize=16)
plt.xlabel('Rows_for_50_Dates')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, axis='y', alpha=0.5)
plt.show()

### The 50-day Moving Average (MA) accurately captures the stock's major trends, particularly demonstrating the MA's sharp drop effectively tracking the severe late-period price crash.

### The AAL stock exhibits higher short-term volatility than AAPL, with its price crossing the 50-day MA frequently, making the MA a less reliable short-term signal.

In [ ]:
df_stock = df_clean[df_clean['Name'] == 'AAP'].copy()
# Calculate 50-day and 200-day Moving Averages
df_stock['MA_Last 50 day'] = df_stock['close'].rolling(window=50).mean()

plt.figure(figsize=(14, 7))
plt.plot(df_stock.index, df_stock['close'], label='Close Price',  color= 'gray', alpha=0.6)
plt.plot(df_stock.index, df_stock['MA_Last 50 day'], label='50-Day MA', color='red')

plt.title('AAP Price with Moving Average', fontsize=16)
plt.xlabel('Rows_for_50_Dates')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, axis='y', alpha=0.5)
plt.show()

### The stock price is contained within a narrow, low-volatility band (STD) for the entire period, with a minor, temporary increase in volatility occurring around the 3500 mark before the price crash.

In [ ]:
df_stock = df_clean[df_clean['Name'] == 'AAP'].copy()
# Calculate 50-day and 200-day Moving Averages
df_stock['STD_Last 50 day'] = df_stock['close'].rolling(window=50).std()
df_stock['STD_Last 200 day'] = df_stock['close'].rolling(window=200).std()

plt.figure(figsize=(14, 7))
plt.plot(df_stock.index, df_stock['close'], label='Close Price',  color= 'gray', alpha=0.6)

plt.plot(df_stock.index, df_stock['STD_Last 50 day'], label='50-Day STD', color='red')
plt.plot(df_stock.index, df_stock['STD_Last 200 day'], label='200-Day STD', color='green')

plt.title('AAP Price with Moving STD', fontsize=16)
plt.xlabel('Rows_for_Dates')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, axis='y', alpha=0.5)
plt.show()

### The stock's volatility (STD) remained consistently low throughout the period, with a minor, temporary increase in risk (around the 3600 mark) preceding the major price crash.

In [ ]:
df_stock = df_clean[df_clean['Name'] == 'AAP'].copy()
# Calculate 50-day and 200-day Moving Averages
df_stock['MA_Last 50 day'] = df_stock['high'].rolling(window=50).mean()
df_stock['MA_Last 200 day'] = df_stock['high'].rolling(window=200).mean()

plt.figure(figsize=(14, 7))
plt.plot(df_stock.index, df_stock['high'], label='High Price',  color= 'gray', alpha=0.6)

plt.plot(df_stock.index, df_stock['MA_Last 50 day'], label='50-Day MA', color='red')
plt.plot(df_stock.index, df_stock['MA_Last 200 day'], label='200-Day MA', color='green')

plt.title('AAP Price with Moving MA', fontsize=16)
plt.xlabel('Rows_for_Dates')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, axis='y', alpha=0.5)
plt.show()

### The Death Cross (50-Day MA crossing below 200-Day MA) is clearly visible around the 3550 mark, effectively signaling the start of the subsequent major downtrend in the stock's price.

In [ ]:
df_stock = df_clean[df_clean['Name'] == 'AAL'].copy()
# Calculate 50-day and 200-day Moving Averages
df_stock['MA_Last 50 day'] = df_stock['high'].rolling(window=50).mean()
df_stock['MA_Last 200 day'] = df_stock['high'].rolling(window=200).mean()

plt.figure(figsize=(14, 7))
plt.plot(df_stock.index, df_stock['high'], label='High Price',  color= 'gray', alpha=0.6)

plt.plot(df_stock.index, df_stock['MA_Last 50 day'], label='50-Day MA', color='red')
plt.plot(df_stock.index, df_stock['MA_Last 200 day'], label='200-Day MA', color='green')

plt.title('AAL Price with Moving MA', fontsize=16)
plt.xlabel('Rows_for_Last_Dates')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, axis='y', alpha=0.5)
plt.show()

### The AAL stock exhibits frequent crossovers between its 50-day and 200-day Moving Averages, indicating significant market volatility and cyclicality over the period shown.

In [ ]:
# 30-day rolling volatility
df_clean['rolling_volatility'] = df_clean['daily_return'].rolling(window=30).std()

plt.figure(figsize=(12,6))
plt.plot(df_clean['date'], df_clean['rolling_volatility'])
plt.title("30-Day Rolling Volatility")
plt.xlabel("Date")
plt.ylabel("Volatility")
plt.show()

## 30-Day Rolling Volatility Insight
### The stock's 30-day volatility (risk) declined dramatically over the period, moving from extreme high levels in 2013 to a very low, stable range by 2018. This suggests the asset became significantly less risky over the five years, though the initial visualization has plotting issues that should be cleaned.

In [ ]:
# 1. Select a few tickers for comparison
top_tickers = ['AAPL','AAP', 'AAL', 'MSFT', 'JPM']

# 2. Pivot data to get Closing prices for selected stocks
df_pivot = df_clean.pivot_table(index='date', columns='Name', values='close')

# 3. Calculate Daily Returns (Percentage Change) for the sample
df_returns_multi = df_pivot[top_tickers].pct_change().dropna()
df_corr = df_returns_multi.corr()

plt.figure(figsize=(10, 8))
# Use Seaborn's heatmap for visualization
sns.heatmap(
    df_corr, 
    annot=True, # Show the correlation values
    cmap='coolwarm', # Color map: red=positive, blue=negative
    fmt=".2f",
    linewidths=.5,
    cbar_kws={'label': 'Correlation Coefficient'}
)
plt.title('Correlation Heatmap of Daily Cleaned Returns ', fontsize=16)
plt.show()

### All stock pairs exhibit positive correlation in their daily returns, with JPM and MSFT showing the strongest relationship (0.41), while AAPL and AAL show the weakest (0.19).

In [ ]:
# Monthly resampling to observe seasonality
monthly = df_clean.set_index('date')['close'].resample('ME').mean()

plt.figure(figsize=(12,6))
plt.plot(monthly)
plt.title("Monthly Average Closing Price")
plt.xlabel("Date")
plt.ylabel("Price")
plt.show()


## Monthly Trend and Cyclicality Analysis

### The monthly resampling successfully **smooths the daily price fluctuations**, revealing a **strong, overall positive long-term trend** for the stock from 2013 to 2018, rising from approximately \$55 to over \$90.

### Crucially, the plot highlights significant **cyclical volatility** (or market corrections), particularly a sharp dip observed between late 2015 and early 2016. This cyclical pattern confirms the **non-linear nature** of the stock's growth, indicating that the price does not just increase steadily but is subject to notable drawdowns.